In [210]:
# imports 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

import pickle

from mapie.regression import MapieRegressor
from mapie.metrics import regression_coverage_score_v2

# Creating Budget & Inspection Projections



In [211]:
df = pd.read_csv('data/df2_with_weather.csv')

In [212]:
df.head()

,DATE1,DAY_OF_WEEK,SITE_NAME,TOWN,WATERBODY,INSPECTOR_ID,PAY_VOL,SHIFT_START,TRAILERS,SHIFT_END,...,PRCP,TMAX,TMIN,DATE,month,year,SHIFT_START_CATEGORY,SHIFT_END_CATEGORY,ID_COUNT,holiday
0,2021-05-28,Fri,Launch Drive Cobbosseecontee,Monmouth,Cobbosseecontee Lake,4771,Paid,1900-01-01 12:00:00,10.0,2023-12-05 18:00:00,...,0.00,72.0,45.0,2021-05-28,5,2021,Afternoon,Evening,55,0
1,2021-05-28,Fri,Whippoorwill Road,Litchfield,Woodbury Pond,4174,Paid,1900-01-01 12:00:00,1.0,2023-12-05 18:00:00,...,0.00,72.0,45.0,2021-05-28,5,2021,Afternoon,Evening,131,0
2,2021-05-28,Fri,Holmes Road Annabessacook,Winthrop,Annabessacook Lake,4769,Paid,1900-01-01 12:00:00,0.0,2023-12-05 18:00:00,...,0.00,72.0,45.0,2021-05-28,5,2021,Afternoon,Evening,56,0
3,2021-05-28,Fri,East Winthrop Cobbosseecontee,Winthrop,Cobbosseecontee Lake,4485,Paid,1900-01-01 12:00:00,1.0,2023-12-05 18:00:00,...,0.00,72.0,45.0,2021-05-28,5,2021,Afternoon,Evening,16,0
4,2021-05-29,Sat,Thorofare Rd,Litchfield,Pleasant Pond,4944,Paid,1900-01-01 07:00:00,1.0,2023-12-05 17:00:00,...,0.02,62.0,45.0,2021-05-29,5,2021,Early Morning,Late Afternoon,57,1


In [213]:
df.dtypes

DATE1                    object
DAY_OF_WEEK              object
SITE_NAME                object
TOWN                     object
WATERBODY                object
INSPECTOR_ID              int64
PAY_VOL                  object
SHIFT_START              object
TRAILERS                float64
SHIFT_END                object
SHIFT_LENGTH              int64
TOTALINSP               float64
PRCP                    float64
TMAX                    float64
TMIN                    float64
DATE                     object
month                     int64
year                      int64
SHIFT_START_CATEGORY     object
SHIFT_END_CATEGORY       object
ID_COUNT                  int64
holiday                   int64
dtype: object

In [214]:
df['DATE'] = pd.to_datetime(df['DATE'])

# Define the start and end dates for Memorial Day weekend through Labor Day weekend
#start_date_2021 = datetime(2021, 5, 28)  # Friday before Memorial Day 
#end_date_2021 = datetime(2021, 9, 6)  # Monday Labor Day

#start_date_2022 = datetime(2022, 5, 27)  # Friday before Memorial Day 
#end_date_2022 = datetime(2022, 9, 5)  # Monday Labor Day

# For now I will just make projections for 2023
start_date_2023 = datetime(2023, 5, 26)  # Friday before Memorial Day 
end_date_2023 = datetime(2023, 9, 4)  # Monday Labor Day

# Create a date range for the specified period
date_range = pd.date_range(start=start_date_2023, end=end_date_2023)

# Create a DataFrame with all combinations of dates and SITE_NAME
new_df = pd.DataFrame([(date, site) for date in date_range for site in df['SITE_NAME'].unique()],
                       columns=['DATE', 'SITE_NAME'])


In [215]:
new_df.head()

,DATE,SITE_NAME
0,2023-05-26,Launch Drive Cobbosseecontee
1,2023-05-26,Whippoorwill Road
2,2023-05-26,Holmes Road Annabessacook
3,2023-05-26,East Winthrop Cobbosseecontee
4,2023-05-26,Thorofare Rd


In [216]:
new_df.shape

(1020, 2)

In [217]:
df.shape

(2321, 22)

In [218]:
# Outer Merge with the original DataFrame (or left with new_df as the left?)
combined_df = pd.merge(new_df, df, how='outer', on=['DATE', 'SITE_NAME'])

# Create a binary column indicating whether there was a match during the merge (this will be used to define which rows are part of the scenarios)
# INSPECTOR_ID is not missing in any shift data so this is a good choice for
combined_df['MATCH'] = pd.notna(combined_df['INSPECTOR_ID'])

In [219]:
combined_df.MATCH.value_counts()

True     2321
False     517
Name: MATCH, dtype: int64

In [220]:
combined_df.head()

,DATE,SITE_NAME,DATE1,DAY_OF_WEEK,TOWN,WATERBODY,INSPECTOR_ID,PAY_VOL,SHIFT_START,TRAILERS,...,PRCP,TMAX,TMIN,month,year,SHIFT_START_CATEGORY,SHIFT_END_CATEGORY,ID_COUNT,holiday,MATCH
0,2023-05-26,Launch Drive Cobbosseecontee,2023-05-26,Fri,Monmouth,Cobbosseecontee Lake,5670.0,Paid,1900-01-01 12:00:00,4.0,...,0.0,66.0,40.0,5.0,2023.0,Afternoon,Evening,116.0,0.0,True
1,2023-05-26,Whippoorwill Road,2023-05-26,Fri,Litchfield,Woodbury Pond,5714.0,Paid,1900-01-01 12:00:00,3.0,...,0.0,66.0,40.0,5.0,2023.0,Afternoon,Evening,23.0,0.0,True
2,2023-05-26,Holmes Road Annabessacook,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,2023-05-26,East Winthrop Cobbosseecontee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,2023-05-26,Thorofare Rd,2023-05-26,Fri,Litchfield,Pleasant Pond,5983.0,Paid,1900-01-01 12:00:00,4.0,...,0.0,66.0,40.0,5.0,2023.0,Afternoon,Evening,33.0,0.0,True


In [221]:
scenario1 = combined_df[combined_df['MATCH'] == False].copy()

In [222]:
scenario1.shape

(517, 23)

In [223]:
scenario1.drop(columns= ['INSPECTOR_ID', 'TRAILERS', 'TOTALINSP', 'DATE1', 'TOWN', 'WATERBODY', 'MATCH'], inplace = True)

In [224]:
scenario1['DATE'] = pd.to_datetime(scenario1['DATE'])

In [225]:
# Filling holiday values
holiday_list_weekends = ['2021-05-31', '2021-05-29', '2021-05-30', '2021-07-04', '2021-07-03', '2021-07-05', '2021-09-06', '2021-09-04', '2021-09-05', '2021-06-19', '2021-06-20',
                '2022-05-28', '2022-05-29', '2022-05-30', '2022-07-02', '2022-07-03', '2022-07-04', '2022-06-19', '2022-09-03', '2022-09-04', '2022-06-18',
                '2023-05-29', '2023-05-27', '2023-05-28', '2023-07-01', '2023-07-02', '2023-07-04', '2023-09-02', '2023-09-03', '2023-09-04', '2023-06-18', '2023-06-17']
scenario1['holiday']= np.where(scenario1['DATE'].isin(holiday_list_weekends), 1, 0)
scenario1

,DATE,SITE_NAME,DAY_OF_WEEK,PAY_VOL,SHIFT_START,SHIFT_END,SHIFT_LENGTH,PRCP,TMAX,TMIN,month,year,SHIFT_START_CATEGORY,SHIFT_END_CATEGORY,ID_COUNT,holiday
2,2023-05-26,Holmes Road Annabessacook,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2023-05-26,East Winthrop Cobbosseecontee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,2023-05-26,Old Kents Hill Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,2023-05-26,Rt 41 North Basin Maranacook,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,2023-05-26,Wilson Pond Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232,2023-09-01,Beach Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1233,2023-09-01,Norcross Point South Basin Maranacook,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1240,2023-09-02,Thorofare Rd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1254,2023-09-03,Rt 41 North Basin Maranacook,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [226]:
# Fill in ID_COUNT with median of ID_COUNT from original df
#scenario1['ID_COUNT'] = np.median(df['ID_COUNT'])

# Fill in month and year from the date category
scenario1['month'] = scenario1['DATE'].dt.month
scenario1['year'] = scenario1['DATE'].dt.year
scenario1['DAY_OF_WEEK'] = scenario1['DATE'].dt.day_name()

# Re-merge with weather data? 

In [227]:
df.dtypes

DATE1                           object
DAY_OF_WEEK                     object
SITE_NAME                       object
TOWN                            object
WATERBODY                       object
INSPECTOR_ID                     int64
PAY_VOL                         object
SHIFT_START                     object
TRAILERS                       float64
SHIFT_END                       object
SHIFT_LENGTH                     int64
TOTALINSP                      float64
PRCP                           float64
TMAX                           float64
TMIN                           float64
DATE                    datetime64[ns]
month                            int64
year                             int64
SHIFT_START_CATEGORY            object
SHIFT_END_CATEGORY              object
ID_COUNT                         int64
holiday                          int64
dtype: object

In [228]:
df.isna().sum()

DATE1                     0
DAY_OF_WEEK               0
SITE_NAME                 0
TOWN                      0
WATERBODY                 0
INSPECTOR_ID              0
PAY_VOL                   0
SHIFT_START               0
TRAILERS                204
SHIFT_END                 0
SHIFT_LENGTH              0
TOTALINSP                 2
PRCP                      0
TMAX                      0
TMIN                      0
DATE                      0
month                     0
year                      0
SHIFT_START_CATEGORY      0
SHIFT_END_CATEGORY        0
ID_COUNT                  0
holiday                   0
dtype: int64

In [229]:
# Fill in values of PAY_VOL, ID_COUNT, SHIFT_START_CATEGORY, SHIFT_END_CATEGORY using the proportions of values from the original df

columns_to_fill = ['PAY_VOL', 'ID_COUNT', 'SHIFT_START', 'SHIFT_END']

# Calculate proportions in the original df
proportions = df[columns_to_fill].count() / len(df)

# Create a function to fill the missing values from selected columns based on proportions
def fill_missing(row, proportions, columns_to_fill):
    for col in columns_to_fill:
        if pd.isna(row[col]):
            # Fill missing value based on the type of column
            if pd.api.types.is_numeric_dtype(df[col].dtype):
                # If the column is numeric, fill with the median value
                row[col] = df[col].median()
            else:
                # If the column is not numeric, fill randomly based on proportions
                choices = df[col].dropna()
                if not choices.empty:
                    if pd.api.types.is_numeric_dtype(choices.dtype):
                        row[col] = np.random.choice(choices, p=proportions[col])
                    else:
                        # Extract only the time part if the column represents datetime values
                        if pd.api.types.is_datetime64_any_dtype(choices.dtype):
                            row[col] = np.random.choice(choices.dt.time)
                        else:
                            row[col] = np.random.choice(choices)
    return row

# Apply the filling function to each row in the target DataFrame
scenario1 = scenario1.apply(lambda row: fill_missing(row, proportions, columns_to_fill), axis=1)

In [230]:
scenario1.head()

,DATE,SITE_NAME,DAY_OF_WEEK,PAY_VOL,SHIFT_START,SHIFT_END,SHIFT_LENGTH,PRCP,TMAX,TMIN,month,year,SHIFT_START_CATEGORY,SHIFT_END_CATEGORY,ID_COUNT,holiday
2,2023-05-26,Holmes Road Annabessacook,Friday,Paid,1900-01-01 12:00:00,2023-12-05 13:00:00,NaN,NaN,NaN,NaN,5,2023,NaN,NaN,55.0,0
3,2023-05-26,East Winthrop Cobbosseecontee,Friday,Paid,1900-01-01 07:00:00,2023-12-05 17:00:00,NaN,NaN,NaN,NaN,5,2023,NaN,NaN,55.0,0
5,2023-05-26,Old Kents Hill Road,Friday,Paid,1900-01-01 13:00:00,2023-12-05 13:00:00,NaN,NaN,NaN,NaN,5,2023,NaN,NaN,55.0,0
6,2023-05-26,Rt 41 North Basin Maranacook,Friday,Paid,1900-01-01 13:00:00,2023-12-05 13:00:00,NaN,NaN,NaN,NaN,5,2023,NaN,NaN,55.0,0
7,2023-05-26,Wilson Pond Road,Friday,Paid,1900-01-01 09:00:00,2023-12-05 13:00:00,NaN,NaN,NaN,NaN,5,2023,NaN,NaN,55.0,0


In [231]:
scenario1.dtypes

DATE                    datetime64[ns]
SITE_NAME                       object
DAY_OF_WEEK                     object
PAY_VOL                         object
SHIFT_START                     object
SHIFT_END                       object
SHIFT_LENGTH                   float64
PRCP                           float64
TMAX                           float64
TMIN                           float64
month                            int64
year                             int64
SHIFT_START_CATEGORY           float64
SHIFT_END_CATEGORY             float64
ID_COUNT                       float64
holiday                          int64
dtype: object

In [232]:
# Use Shift_start and Shift_end to create shift_start_category, shift_end category and shift_length
scenario1['SHIFT_START'] = pd.to_datetime(scenario1['SHIFT_START']).dt.time
scenario1['SHIFT_END'] = pd.to_datetime(scenario1['SHIFT_END']).dt.time

scenario1['SHIFT_START_CATEGORY'] = pd.cut(pd.to_datetime(scenario1['SHIFT_START'].astype(str), format='%H:%M:%S').dt.hour,
                                    bins=[-1, 8, 11, 14, 17, 24],
                                    labels=['Early Morning', 'Morning', 'Afternoon', 'Late Afternoon', 'Evening'])

scenario1['SHIFT_END_CATEGORY'] = pd.cut(pd.to_datetime(scenario1['SHIFT_END'].astype(str), format='%H:%M:%S').dt.hour,
                                    bins=[-1, 8, 11, 14, 17, 19, 24],
                                    labels=['Early Morning', 'Morning', 'Afternoon', 'Late Afternoon', 'Evening', 'Night'])

scenario1['SHIFT_LENGTH'] = (pd.to_datetime(scenario1['SHIFT_END'].astype(str), format='%H:%M:%S') - pd.to_datetime(scenario1['SHIFT_START'].astype(str), format='%H:%M:%S')).dt.total_seconds() / 60


In [233]:
# I can create a function to complete the above column transformations to repeat this process for other scenarios

def create_shift_features(df, start_col='SHIFT_START', end_col='SHIFT_END'):
    # Convert to datetime and extract time
    df[start_col] = pd.to_datetime(df[start_col]).dt.time
    df[end_col] = pd.to_datetime(df[end_col]).dt.time

    # Create shift_start_category
    df['SHIFT_START_CATEGORY'] = pd.cut(pd.to_datetime(df[start_col].astype(str), format='%H:%M:%S').dt.hour,
                                        bins=[-1, 8, 11, 14, 17, 24],
                                        labels=['Early Morning', 'Morning', 'Afternoon', 'Late Afternoon', 'Evening'])

    # Create shift_end_category
    df['SHIFT_END_CATEGORY'] = pd.cut(pd.to_datetime(df[end_col].astype(str), format='%H:%M:%S').dt.hour,
                                      bins=[-1, 8, 11, 14, 17, 19, 24],
                                      labels=['Early Morning', 'Morning', 'Afternoon', 'Late Afternoon', 'Evening', 'Night'])

    # Create shift_length
    df['SHIFT_LENGTH'] = (pd.to_datetime(df[end_col].astype(str), format='%H:%M:%S') - 
                          pd.to_datetime(df[start_col].astype(str), format='%H:%M:%S')).dt.total_seconds() / 60

    return df


In [234]:
scenario1.head()

,DATE,SITE_NAME,DAY_OF_WEEK,PAY_VOL,SHIFT_START,SHIFT_END,SHIFT_LENGTH,PRCP,TMAX,TMIN,month,year,SHIFT_START_CATEGORY,SHIFT_END_CATEGORY,ID_COUNT,holiday
2,2023-05-26,Holmes Road Annabessacook,Friday,Paid,12:00:00,13:00:00,60.0,NaN,NaN,NaN,5,2023,Afternoon,Afternoon,55.0,0
3,2023-05-26,East Winthrop Cobbosseecontee,Friday,Paid,07:00:00,17:00:00,600.0,NaN,NaN,NaN,5,2023,Early Morning,Late Afternoon,55.0,0
5,2023-05-26,Old Kents Hill Road,Friday,Paid,13:00:00,13:00:00,0.0,NaN,NaN,NaN,5,2023,Afternoon,Afternoon,55.0,0
6,2023-05-26,Rt 41 North Basin Maranacook,Friday,Paid,13:00:00,13:00:00,0.0,NaN,NaN,NaN,5,2023,Afternoon,Afternoon,55.0,0
7,2023-05-26,Wilson Pond Road,Friday,Paid,09:00:00,13:00:00,240.0,NaN,NaN,NaN,5,2023,Morning,Afternoon,55.0,0


In [235]:
scenario1['DATE'] = pd.to_datetime(scenario1['DATE'])

In [236]:
scenario1.drop(columns=['PRCP', 'TMAX', 'TMIN'], inplace=True)

In [237]:
# Read in the complete weather data and merge with scenario1

# Reading in weather data
weather = pd.read_csv('data/weather.csv')
weather.drop(columns=['STATION', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DAPR', 'MDPR', 'SNWD', 'WT01', 'WT03', 'WT04', 'WT05', 'WT06', 'WT11', 'SNOW', 'TOBS'], inplace=True)
weather = weather[weather['NAME']== 'WINTHROP, ME US']
weather.fillna(method='bfill', inplace=True)

In [238]:
weather.head()

,NAME,DATE,PRCP,TMAX,TMIN
0,"WINTHROP, ME US",2021-01-01,0.00,42.0,20.0
1,"WINTHROP, ME US",2021-01-03,0.00,34.0,26.0
2,"WINTHROP, ME US",2021-01-04,0.00,34.0,26.0
3,"WINTHROP, ME US",2021-01-05,0.04,35.0,28.0
4,"WINTHROP, ME US",2021-01-06,0.03,32.0,25.0


In [239]:
weather['DATE'] = pd.to_datetime(weather['DATE'])
#weather['DATE'] = weather['DATE'].dt.strftime('%Y/%m/%d')
weather['DATE']=pd.to_datetime(weather['DATE'])
weather.drop(columns=['NAME'], inplace=True)

scenario1 = pd.merge(left=scenario1, right=weather, how='left', on='DATE')

In [240]:
scenario1.head()

,DATE,SITE_NAME,DAY_OF_WEEK,PAY_VOL,SHIFT_START,SHIFT_END,SHIFT_LENGTH,month,year,SHIFT_START_CATEGORY,SHIFT_END_CATEGORY,ID_COUNT,holiday,PRCP,TMAX,TMIN
0,2023-05-26,Holmes Road Annabessacook,Friday,Paid,12:00:00,13:00:00,60.0,5,2023,Afternoon,Afternoon,55.0,0,0.0,66.0,40.0
1,2023-05-26,East Winthrop Cobbosseecontee,Friday,Paid,07:00:00,17:00:00,600.0,5,2023,Early Morning,Late Afternoon,55.0,0,0.0,66.0,40.0
2,2023-05-26,Old Kents Hill Road,Friday,Paid,13:00:00,13:00:00,0.0,5,2023,Afternoon,Afternoon,55.0,0,0.0,66.0,40.0
3,2023-05-26,Rt 41 North Basin Maranacook,Friday,Paid,13:00:00,13:00:00,0.0,5,2023,Afternoon,Afternoon,55.0,0,0.0,66.0,40.0
4,2023-05-26,Wilson Pond Road,Friday,Paid,09:00:00,13:00:00,240.0,5,2023,Morning,Afternoon,55.0,0,0.0,66.0,40.0


In [241]:
scenario1.isna().sum()

DATE                    0
SITE_NAME               0
DAY_OF_WEEK             0
PAY_VOL                 0
SHIFT_START             0
SHIFT_END               0
SHIFT_LENGTH            0
month                   0
year                    0
SHIFT_START_CATEGORY    0
SHIFT_END_CATEGORY      0
ID_COUNT                0
holiday                 0
PRCP                    0
TMAX                    0
TMIN                    0
dtype: int64

In [253]:
# Define X for scenario data
X = scenario1.drop(columns=['DATE', 'SHIFT_START', 'SHIFT_END'])


# Specify categorical columns
categorical_columns = ['SITE_NAME', 'DAY_OF_WEEK', 'month', 'year', 'PAY_VOL', 'SHIFT_START_CATEGORY', 'SHIFT_END_CATEGORY']

# Pre-processing steps
with open('models/one_hot_encoder.pkl', 'rb') as file:
    oh = pickle.load(file)

X_transformed = oh.transform(X[categorical_columns])

C:\Users\kaitl\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [254]:
# Load pickled model
with open('models/grad_1.pkl', 'rb') as file:
    model = pickle.load(file)

In [262]:
# make predictions on scenario data
predictions = model.predict(X_transformed)

# Ensure there are no negative values
predictions = np.maximum(predictions, 0)

In [263]:
scenario1['Predictions'] = predictions

In [264]:
# Use Boostrapping/ resampling to create a prediction interval
# Source: https://otexts.com/fpp2/aggregates.html

# Number of bootstrap samples
num_bootstraps = 1000

# Create an array to store the summed predictions
summed_predictions = np.zeros(num_bootstraps)

# Perform bootstrapping
for i in range(num_bootstraps):
    # Randomly sample with replacement from the predicted values
    bootstrap_sample = np.random.choice(predictions, size=len(predictions), replace=True)
    
    # Sum the predictions for the bootstrap sample
    summed_predictions[i] = np.sum(bootstrap_sample)

# Calculate prediction intervals
lower_bound = np.percentile(summed_predictions, 2.5)
upper_bound = np.percentile(summed_predictions, 97.5)

print(f"Prediction Interval if all missed shifts were covered: ({lower_bound}, {upper_bound})")

Prediction Interval if all missed shifts were covered: (5035.640541858166, 5855.856336244167)


In [265]:
scenario1.head()

,DATE,SITE_NAME,DAY_OF_WEEK,PAY_VOL,SHIFT_START,SHIFT_END,SHIFT_LENGTH,month,year,SHIFT_START_CATEGORY,SHIFT_END_CATEGORY,ID_COUNT,holiday,PRCP,TMAX,TMIN,Predictions
0,2023-05-26,Holmes Road Annabessacook,Friday,Paid,12:00:00,13:00:00,60.0,5,2023,Afternoon,Afternoon,55.0,0,0.0,66.0,40.0,1.553198
1,2023-05-26,East Winthrop Cobbosseecontee,Friday,Paid,07:00:00,17:00:00,600.0,5,2023,Early Morning,Late Afternoon,55.0,0,0.0,66.0,40.0,9.716831
2,2023-05-26,Old Kents Hill Road,Friday,Paid,13:00:00,13:00:00,0.0,5,2023,Afternoon,Afternoon,55.0,0,0.0,66.0,40.0,0.000000
3,2023-05-26,Rt 41 North Basin Maranacook,Friday,Paid,13:00:00,13:00:00,0.0,5,2023,Afternoon,Afternoon,55.0,0,0.0,66.0,40.0,2.824279
4,2023-05-26,Wilson Pond Road,Friday,Paid,09:00:00,13:00:00,240.0,5,2023,Morning,Afternoon,55.0,0,0.0,66.0,40.0,0.000000


In [266]:
scenario1.describe()

,SHIFT_LENGTH,month,year,ID_COUNT,holiday,PRCP,TMAX,TMIN,Predictions
count,517.000000,517.000000,517.0,517.0,517.000000,517.000000,517.000000,517.000000,517.000000
mean,389.299807,6.943907,2023.0,55.0,0.015474,0.208375,77.765957,59.098646,10.544430
std,253.989206,0.945587,0.0,0.0,0.123547,0.417303,8.070028,6.503721,9.179875
min,-362.000000,5.000000,2023.0,55.0,0.000000,0.000000,52.000000,40.000000,0.000000
25%,240.000000,6.000000,2023.0,55.0,0.000000,0.000000,72.000000,54.000000,2.184130
50%,360.000000,7.000000,2023.0,55.0,0.000000,0.010000,78.000000,59.000000,8.460187
75%,600.000000,8.000000,2023.0,55.0,0.000000,0.150000,83.000000,65.000000,16.844493
max,865.000000,9.000000,2023.0,55.0,1.000000,1.970000,95.000000,71.000000,42.148488


In [267]:
scenario1.shape

(517, 17)

In [ ]:
# Now I want to cover ~100 shifts prioritizing top or top few sites
# Then compare that to covering ~100 shifts bottom few sites

# Get the predictions of number of inspections
# Add the total number of hours/ shift length
# Chart this with the potential labor cost

In [ ]:
# Number of bootstrap samples
num_bootstraps = 1000

# Create an array to store the summed predictions
summed_predictions = np.zeros(num_bootstraps)

# Perform bootstrapping
for i in range(num_bootstraps):
    # Randomly sample with replacement from the predicted values
    bootstrap_sample = np.random.choice(predictions, size=len(predictions), replace=True)
    
    # Sum the predictions for the bootstrap sample
    summed_predictions[i] = np.sum(bootstrap_sample)

# Calculate prediction intervals
lower_bound = np.percentile(summed_predictions, 2.5)
upper_bound = np.percentile(summed_predictions, 97.5)

print(f"Prediction Interval if all missed shifts were covered: ({lower_bound}, {upper_bound})")

In [ ]:
# Scenario 2: Instead of random filling based on proportions, random fill the ID_COUNT but put in all paid, and enter morning shift starts with a standard shift_length

# Scenario 3: Fill with later start

# Scenario 4: Morning shifts, but shorter shifts

# Scenario 5: Later shifts, shorter shifts

In [ ]:
# Can i create a bigger function to run through each scenario and then create a dataframe of the expected hours, shifts, cost etc?